In [2]:
import numpy as np
import pandas as pd
from configs.paths import OUTPUT_ROOT

In [15]:
STATE_NAME = 'alabama'
df_embb = pd.read_csv(f'{OUTPUT_ROOT}/{STATE_NAME}/{STATE_NAME}-embeddings.csv')
df_filter = pd.read_csv(f'{OUTPUT_ROOT}/{STATE_NAME}/{STATE_NAME}-filtrado.csv')
output_path = f'{OUTPUT_ROOT}/{STATE_NAME}/pre-processing/'

# create_input(df_embb, df_filter, output_path)

In [20]:
def generate_sequences(places_visited):
    # Early return for short sequences
    if len(places_visited) <= 5:
        return 0

    # Convert to NumPy array for better performance
    places_array = np.array(places_visited)

    # Calculate how many complete sequences we can have
    num_sequences = (len(places_array) - 1) // 9 + 1

    # Pre-allocate the result array
    result = np.zeros((num_sequences, 10), dtype=int)

    for i in range(num_sequences):
        start_idx = i * 9

        if len(places_array) >= start_idx + 10:
            # We have enough elements for a full sequence
            result[i] = places_array[start_idx:start_idx + 10]
        else:
            # We need padding
            seq_len = len(places_array) - start_idx
            padding = np.full(10 - seq_len, -1, dtype=int)
            result[i] = np.concatenate([padding, places_array[start_idx:]])

    # Convert back to a list of lists for compatibility
    return result.tolist()

In [16]:
output_folder = output_path

category_path = f'{output_folder}/category.csv'
next_path = f'{output_folder}/next_checkin.csv'

#Processing Category
df_embb.set_index('placeid', inplace=True)
df_embb.to_csv(category_path)

In [19]:
#Processing Next
df_filter['datetime'] = pd.to_datetime(df_filter['datetime'])
df_filter.sort_values(by=['userid', 'datetime'])
gpp_users = df_filter.groupby('userid').agg(list).reset_index()

checking_by_user = gpp_users[['userid']].copy()
checking_by_user['visit_sequence'] = gpp_users['placeid'].apply(generate_sequences, axis=1)
checking_by_user = checking_by_user[checking_by_user['visit_sequence'] != 0]
print(f'checkins_sequence_by_user shape{checking_by_user.shape}\n')


checking_by_user.set_index('userid', inplace=True)
user_ids = checking_by_user.index.unique()
print(f'total users: {len(user_ids)}\n')

embb = df_embb.drop(columns=['category'])
list_embb = np.zeros(100)

embb.iloc[-1] = list_embb.tolist() + [0]
embb.iloc[-1,'category'] = 'None'







NameError: name 'generate_sequences' is not defined